In [1]:
import pandas as pd
import numpy as np
import os

# processed 폴더 경로
processed_dir = r'D:\python\코드잇 스프린트 데분\codeit-sprint-projects\sprint-missions\스프린트 미션 12\data\processed'

# 파일 로드
train = pd.read_csv(os.path.join(processed_dir, 'train_processed.csv'))
test = pd.read_csv(os.path.join(processed_dir, 'test_processed.csv'))

# Feature list
feature_columns = [
    'holiday', 'workingday', 'weather',
    'temp', 
    'humidity_mapped', 'windspeed_mapped',
    'year', 'month', 'hour', 'weekday'
]

# Target
target_column = 'log_count'

# X, y 나누기
X_train = train[feature_columns]
y_train = train[target_column]

X_test = test[feature_columns]  # 나중에 submission 용


In [20]:
# 파일 로드
train2 = pd.read_csv(os.path.join(processed_dir, 'train_processed2.csv'))
test2 = pd.read_csv(os.path.join(processed_dir, 'test_processed2.csv'))

# Feature list
feature_columns = [
    'holiday', 'workingday', 'weather',
    'temp', 
    'humidity_mapped', 'windspeed_mapped',
    'year', 'month', 'hour', 'weekday'
]


# X, y 나누기
X_train2 = train2[feature_columns]
y_train2 = train2[target_column]

X_test2 = test2[feature_columns]  # 나중에 submission 용


In [21]:
from sklearn.preprocessing import LabelEncoder

# 인코딩할 컬럼
cat_features = ['humidity_mapped', 'windspeed_mapped', 'weekday']

# 각 컬럼마다 LabelEncoder 적용
for col in cat_features:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train2[col])
    X_test[col] = le.transform(X_test2[col])  # Ridge, Lasso, LinearRegression 모델은 라벨 인코딩을 해야 하기 때문에 카테고리 컬럼을 인코딩함.


C:\Users\ypsoo\AppData\Local\Temp\ipykernel_30696\2038018542.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = le.fit_transform(X_train2[col])
C:\Users\ypsoo\AppData\Local\Temp\ipykernel_30696\2038018542.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[col] = le.transform(X_test2[col])  # Ridge, Lasso, LinearRegression 모델은 라벨 인코딩을 해야 하기 때문에 카테고리 컬럼을 인코딩함.
C:\Users\ypsoo\AppData\Local\Temp\ipykernel_30696\2038018542.py:9: SettingWithCopyWarning: 
A value is trying to be set on a c

In [15]:
# 필요 라이브러리
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
import xgboost as xgb

# 1️⃣ Train-validation split
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)
# XGBRegressor 개선 버전 (튜닝 전 약간 다듬은 형태)
xgb_model = xgb.XGBRegressor(
    n_estimators=3000,          # 트리 수 늘림 (early_stopping 안 쓰니 그냥 다 돌려도 됨)
    learning_rate=0.03,         # learning_rate 낮추기 (성능 개선 효과 큼)
    max_depth=5,                # 살짝 낮춤 (6 → 5)
    min_child_weight=3,         # 과적합 방지
    subsample=0.8,              # 전체 샘플 중 80%만 사용 (과적합 방지)
    colsample_bytree=0.8,       # 전체 feature 중 80%만 사용 (과적합 방지)
    random_state=42,
    tree_method='hist',
    verbosity=1
)

# 학습
xgb_model.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    verbose=100
)

# validation 예측
y_val_pred = xgb_model.predict(X_val)

# RMSLE 계산
rmsle = np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_val_pred)))
print(f'✅ 개선된 XGBoost Validation RMSLE: {rmsle:.5f}')



[0]	validation_0-rmse:1.39978
[100]	validation_0-rmse:0.42048
[200]	validation_0-rmse:0.31419
[300]	validation_0-rmse:0.29566
[400]	validation_0-rmse:0.28712
[500]	validation_0-rmse:0.28209
[600]	validation_0-rmse:0.27840
[700]	validation_0-rmse:0.27599
[800]	validation_0-rmse:0.27406
[900]	validation_0-rmse:0.27263
[1000]	validation_0-rmse:0.27159
[1100]	validation_0-rmse:0.27062
[1200]	validation_0-rmse:0.27039
[1300]	validation_0-rmse:0.26980
[1400]	validation_0-rmse:0.26975
[1500]	validation_0-rmse:0.26965
[1600]	validation_0-rmse:0.26973
[1700]	validation_0-rmse:0.26938
[1800]	validation_0-rmse:0.26913
[1900]	validation_0-rmse:0.26899
[2000]	validation_0-rmse:0.26903
[2100]	validation_0-rmse:0.26904
[2200]	validation_0-rmse:0.26909
[2300]	validation_0-rmse:0.26922
[2400]	validation_0-rmse:0.26922
[2500]	validation_0-rmse:0.26928
[2600]	validation_0-rmse:0.26938
[2700]	validation_0-rmse:0.26957
[2800]	validation_0-rmse:0.26963
[2900]	validation_0-rmse:0.26997
[2999]	validation_0-rm

In [22]:
# 필요 라이브러리
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
import xgboost as xgb

# 1️⃣ Train-validation split
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train2, y_train2, test_size=0.2, random_state=42
)
# XGBRegressor 개선 버전 (튜닝 전 약간 다듬은 형태)
xgb_model = xgb.XGBRegressor(
    n_estimators=3000,          # 트리 수 늘림 (early_stopping 안 쓰니 그냥 다 돌려도 됨)
    learning_rate=0.03,         # learning_rate 낮추기 (성능 개선 효과 큼)
    max_depth=5,                # 살짝 낮춤 (6 → 5)
    min_child_weight=3,         # 과적합 방지
    subsample=0.8,              # 전체 샘플 중 80%만 사용 (과적합 방지)
    colsample_bytree=0.8,       # 전체 feature 중 80%만 사용 (과적합 방지)
    random_state=42,
    tree_method='hist',
    verbosity=1
)

# 학습
xgb_model.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    verbose=100
)

# validation 예측
y_val_pred = xgb_model.predict(X_val)

# RMSLE 계산
rmsle = np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_val_pred)))
print(f'✅ 개선된 XGBoost Validation RMSLE: {rmsle:.5f}')



[0]	validation_0-rmse:1.39977


[100]	validation_0-rmse:0.42047
[200]	validation_0-rmse:0.31434
[300]	validation_0-rmse:0.29569
[400]	validation_0-rmse:0.28754
[500]	validation_0-rmse:0.28288
[600]	validation_0-rmse:0.27954
[700]	validation_0-rmse:0.27707
[800]	validation_0-rmse:0.27518
[900]	validation_0-rmse:0.27411
[1000]	validation_0-rmse:0.27282
[1100]	validation_0-rmse:0.27228
[1200]	validation_0-rmse:0.27189
[1300]	validation_0-rmse:0.27125
[1400]	validation_0-rmse:0.27090
[1500]	validation_0-rmse:0.27082
[1600]	validation_0-rmse:0.27074
[1700]	validation_0-rmse:0.27071
[1800]	validation_0-rmse:0.27033
[1900]	validation_0-rmse:0.27051
[2000]	validation_0-rmse:0.27036
[2100]	validation_0-rmse:0.27065
[2200]	validation_0-rmse:0.27090
[2300]	validation_0-rmse:0.27076
[2400]	validation_0-rmse:0.27103
[2500]	validation_0-rmse:0.27104
[2600]	validation_0-rmse:0.27095
[2700]	validation_0-rmse:0.27110
[2800]	validation_0-rmse:0.27134
[2900]	validation_0-rmse:0.27153
[2999]	validation_0-rmse:0.27148
✅ 개선된 XGBoost Valid